In [1]:
from torch.utils.data import Dataset
from torchvision import transforms, datasets
from torch import no_grad
import torch.nn as nn
from PIL import Image
import os
from torch.utils.data import DataLoader
import torch

In [2]:
class CatDogSnake(Dataset):
    def __init__(self, root_dir, transform):
        self.root_dir = root_dir
        self.transform = transform

        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        self.image_path = []
        self.labels = []

        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            for img in os.listdir(cls_path):
                self.image_path.append(os.path.join(cls_path,img))
                self.labels.append(self.class_to_idx[cls])

    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self, idx):
        img_path = self.image_path[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(img)

        return image, label

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4981, 0.4601, 0.4107],
                         std=[0.2214, 0.2159, 0.2146])
])

dataset = CatDogSnake(r"Cat_Dog_Snake", transform=transform)

In [4]:
print(len(dataset))
img, label = dataset[0]
print(label)

3000
0


In [5]:
# Check a few images
for i in [0, 1000, 1500,2500]:
    img, _ = dataset[i]
    print(f'Image {i}: {img.shape}')
    print(f'Image {i}: {type(img)}')

Image 0: torch.Size([3, 224, 224])
Image 0: <class 'torch.Tensor'>
Image 1000: torch.Size([3, 224, 224])
Image 1000: <class 'torch.Tensor'>
Image 1500: torch.Size([3, 224, 224])
Image 1500: <class 'torch.Tensor'>
Image 2500: torch.Size([3, 224, 224])
Image 2500: <class 'torch.Tensor'>


In [6]:
# # Load dataset WITHOUT normalization first
# dataset = datasets.ImageFolder(
#     root=r"Cat_Dog_Snake",
#     transform=transforms.ToTensor()
# )

# loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)

# mean = 0.
# std = 0.
# total_images = 0

# for images, _ in loader:
#     batch_samples = images.size(0)  # batch size
#     images = images.view(batch_samples, images.size(1), -1)  # flatten H*W

#     mean += images.mean(2).sum(0)
#     std += images.std(2).sum(0)
#     total_images += batch_samples

# mean /= total_images
# std /= total_images

# print("Mean:", mean)
# print("Std:", std)


In [7]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = int(0.15 * len(dataset))

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

In [8]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [9]:
for images, labels in train_loader:
    print(f"Batch of images shape: {images.shape}")
    print(f"Batch of labels shape: {labels.shape}")
    print(f"Batch of labels: {labels}")
    break

Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels shape: torch.Size([32])
Batch of labels: tensor([2, 2, 2, 0, 2, 0, 1, 0, 1, 0, 1, 2, 2, 2, 0, 2, 2, 0, 2, 2, 1, 2, 1, 0,
        0, 0, 2, 2, 2, 1, 2, 1])


In [10]:
batch_count = 0
total_images = 0

for images, labels in train_loader:
    batch_count += 1
    total_images += len(images)

    if batch_count >= 65:
        print(f"Batch {batch_count}: {len(images)} images")

print(f"\nTotal batches in one epoch: {batch_count}")
print(f"TOtal images seen: {total_images}")

Batch 65: 32 images
Batch 66: 20 images

Total batches in one epoch: 66
TOtal images seen: 2100


In [11]:
print(f"Train: {len(train_loader)} batches")
print(f"Val: {len(val_loader)} batches")
print(f"Test: {len(test_loader)} batches")

for name, loader in [("Train",train_loader), ("Val",val_loader), ("Test", test_loader)]:
    images, labels = next(iter(loader))
    print(f"{name} batch: {images.shape}")


Train: 66 batches
Val: 15 batches
Test: 15 batches
Train batch: torch.Size([32, 3, 224, 224])
Val batch: torch.Size([32, 3, 224, 224])
Test batch: torch.Size([32, 3, 224, 224])


In [12]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128*28*28, 128)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.flatten(x)

        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [13]:
model = CNNClassifier()
print(model)

CNNClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=100352, out_features=128, bias=True)
  (relu4): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
)


In [14]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.0005)

In [15]:
def train_epoch(model, train_loader, loss_function, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        if batch_idx % 100 == 0 and batch_idx > 0:
            avg_loss = running_loss/100
            accuracy = 100. * correct / total
            print(f" [{batch_idx * 64}/{60000}]"
                  f"Loss: {avg_loss:.3f} | Accuracy: {accuracy:.1f}%")
            running_loss = 0

In [16]:
def evaluation(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            _,predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    return 100. * correct / total

In [17]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f"\nEpoch: {epoch+1}")
    train_epoch(model, train_loader, loss_function, optimizer)
    accuracy = evaluation(model, val_loader)
    print(f"Test Accuracy: {accuracy:.2f}%")


Epoch: 1


KeyboardInterrupt: 